# Project Title: Find a place to open a restaurant

## Data:
* Latitude, longitude, and estimated population of 2015 data - Download csv data from United States Zip Codes.org
* Location and surrounding data - From Foursquare

## Criteria:
* Only Maryland, US data
* Sort by highest estimated population in 2015 from IRS


In [1]:
# Import the libraries we need in the project
# Import requests #Library to handle requests

import pandas as pd #Library for data analysis
import numpy as np #Library to handle data in a vectorized manner
import json
import random #Library for random number generation

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0
import folium as folium

# Libraries for displaying images
from IPython.display import Image
from IPython.core.display import HTML

print('Libraries imported.')


Libraries imported.


In [2]:
# Use pandas read_cvs to read the postal data
postalData = pd.read_csv('zip_code_database.csv')
postalData.head()

,zip,type,decommissioned,primary_city,acceptable_cities,unacceptable_cities,state,county,timezone,area_codes,world_region,country,latitude,longitude,irs_estimated_population_2015
0,501,UNIQUE,0,Holtsville,NaN,I R S Service Center,NY,Suffolk County,America/New_York,631,NaN,US,40.81,-73.04,562
1,544,UNIQUE,0,Holtsville,NaN,Irs Service Center,NY,Suffolk County,America/New_York,631,NaN,US,40.81,-73.04,0
2,601,STANDARD,0,Adjuntas,NaN,"Colinas Del Gigante, Jard De Adjuntas, Urb San...",PR,Adjuntas Municipio,America/Puerto_Rico,"787,939",NaN,US,18.16,-66.72,0
3,602,STANDARD,0,Aguada,NaN,"Alts De Aguada, Bo Guaniquilla, Comunidad Las ...",PR,Aguada Municipio,America/Puerto_Rico,"787,939",NaN,US,18.38,-67.18,0
4,603,STANDARD,0,Aguadilla,Ramey,"Bda Caban, Bda Esteves, Bo Borinquen, Bo Ceiba...",PR,Aguadilla Municipio,America/Puerto_Rico,787,NaN,US,18.43,-67.15,0


In [3]:
# Now, clean up the data and keep only the column and data we need. The target area is Baltimore County, Maryland, USA.
MDData = postalData.drop(columns=['decommissioned', 'unacceptable_cities', 'timezone', 'world_region'])
MDData.sort_values(by='irs_estimated_population_2015', ascending=False)

,zip,type,primary_city,acceptable_cities,state,county,area_codes,country,latitude,longitude,irs_estimated_population_2015
26692,60629,STANDARD,Chicago,Bedford Park,IL,Cook County,"312,773,872",US,41.78,-87.71,114420
4118,11220,STANDARD,Brooklyn,NaN,NY,Kings County,718,US,40.64,-74.02,111430
34023,77449,STANDARD,Katy,Park Row,TX,Harris County,"281,346,832",US,29.84,-95.73,109280
3135,8701,STANDARD,Lakewood,NaN,NJ,Ocean County,"732,848,908",US,40.09,-74.21,105330
34065,77494,STANDARD,Katy,Park Row,TX,Fort Bend County,"281,832",US,29.74,-95.83,104450
...,...,...,...,...,...,...,...,...,...,...,...
10029,24042,UNIQUE,Roanoke,NaN,VA,Roanoke City,540,US,37.27,-79.95,0
10030,24043,UNIQUE,Roanoke,NaN,VA,Roanoke City,540,US,37.27,-79.95,0
10031,24044,UNIQUE,Roanoke,NaN,VA,Roanoke City,540,US,37.27,-79.94,0
10032,24045,UNIQUE,Roanoke,NaN,VA,Roanoke City,540,US,37.27,-79.94,0


In [4]:
# Clean up the data and filter to Baltimore County, Maryland, USA. And the population is above 20,000

MDData = MDData[((MDData['type'] == 'UNIQUE') | (MDData['type'] == 'STANDARD')) & (MDData['state'] == 'MD')]  
#MDData = MDData[(MDData['state'] == 'MD') & (MDData['irs_estimated_population_2015'] > 20000)]
MDData.sort_values(by='irs_estimated_population_2015', ascending=False, inplace=True)
MDData.groupby(['county', 'primary_city'])
MDData

,zip,type,primary_city,acceptable_cities,state,county,area_codes,country,latitude,longitude,irs_estimated_population_2015
8791,20906,STANDARD,Silver Spring,Aspen Hill,MD,Montgomery County,"301,240",US,39.09,-77.06,68290
8769,20878,STANDARD,Gaithersburg,"Darnestown, N Potomac, No Potomac, North Potomac",MD,Montgomery County,"240,301",US,39.12,-77.25,62930
8929,21234,STANDARD,Parkville,Baltimore,MD,Baltimore County,"410,443",US,39.38,-76.55,62620
8765,20874,STANDARD,Germantown,Darnestown,MD,Montgomery County,240,US,39.17,-77.26,59300
8873,21122,STANDARD,Pasadena,"Lake Shore, Millersville, Riviera Beach",MD,Anne Arundel County,"410,443",US,39.11,-76.55,57620
...,...,...,...,...,...,...,...,...,...,...,...
9079,21709,UNIQUE,Frederick,NaN,MD,Frederick County,240,US,39.42,-77.41,0
8945,21268,UNIQUE,Baltimore,NaN,MD,Baltimore City,410,US,39.21,-76.72,0
8947,21273,UNIQUE,Baltimore,NaN,MD,Baltimore City,410,US,39.30,-76.61,0
9073,21690,UNIQUE,Chestertown,NaN,MD,Kent County,410,US,39.21,-76.07,0


In [5]:
MDData.shape

(481, 11)